# Create New Cycle

This notebook creates a new active cycle for your Risk Modeling workflow. 

1. Archives the current active cycle (if any)
2. Creates a new cycle from the template
3. Registers stages and steps in the database

**Note:** Only one cycle can be active at a time.

In [1]:
# Import modules directly
from helpers import cycle, ux
from helpers.context import WorkContext
from helpers.step import Step
from helpers import constants

## Look into current cycles (if any)

In [2]:
ux.subheader("Current Cycles")

cycles_df = cycle.get_cycle_status()
if not cycles_df.empty:
    ux.dataframe(cycles_df, max_rows=10)
else:
    print("No cycles found")

# Check for active cycle
active_cycle = cycle.get_active_cycle()
if active_cycle:
    ux.error(f"\nActive cycle exists: {active_cycle['cycle_name']}")

    if ux.yes_no("This will be archived when creating a new cycle. Do you wish to proceed?"):
        ux.warning(f"Creation of new cycle below will archive {active_cycle['cycle_name']}.")
    else:
        ux.error("Execution stopped by user.")
        raise Exception
else:
    ux.success("No active cycle - ready to create new cycle")

This will be archived when creating a new cycle. Do you wish to proceed? (y/n):  n


Exception: 

## New Cycle Name

In [ ]:
new_cycle_name = ux.text_input("Enter cycle name to proceed", placeholder = constants.CYCLE_NAME_RULES['example'], validation=cycle.validate_cycle_name)
if new_cycle_name is None:
    ux.error("Text input was canceled, execution stopped.")
    exit
else:
    ux.info(f"Cycle name: {new_cycle_name}!")
    ux.warning("The cycle has not been created yet. It will be created in the cells below.")

### Creating the Cycle

In [ ]:
# Creating the cycle
ux.header(f"Creating Cycle: {new_cycle_name}")

# Create the cycle
success = cycle.create_cycle(new_cycle_name)

if success:
    ux.header("CYCLE CREATED SUCCESSFULLY")
else:
    ux.error("Failed to create cycle")

In [ ]:
# Show directory structure
from pathlib import Path
active_dir = Path(f"/home/jovyan/workspace/workflows/Active_{new_cycle_name}")

if active_dir.exists():
    ux.subheader("Directory Structure")
    print(f"📁 {active_dir}\n")
    
    for item in sorted(active_dir.iterdir()):
        if item.is_dir():
            print(f"  📂 {item.name}/")
            if item.name == 'notebooks':
                for subitem in sorted(item.iterdir()):
                    if subitem.is_dir():
                        print(f"      📂 {subitem.name}/")

In [ ]:
ux.header("Verification")

# Check database
active_cycle = cycle.get_active_cycle()
if active_cycle:
    ux.success(f"Active cycle in database: {active_cycle['cycle_name']}")
else:
    ux.error("No active cycle in database")

# Check filesystem
from pathlib import Path
workflows_path = Path('/home/jovyan/workspace/workflows')
active_dirs = [d for d in workflows_path.iterdir() if d.is_dir() and d.name.startswith('Active_')]

if active_dirs:
    ux.success(f"Active directory found: {active_dirs[0].name}")
else:
    ux.error("No active directory found")

# Show registered steps
if active_cycle:
    progress_df = cycle.get_cycle_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        ux.subheader(f"The cycle has {len(progress_df)} steps")
        ux.step_progress(progress_df)
    else:
        ux.warning("No stages and steps - template may be empty")

---